In [1]:
import psycopg2
import urllib
import json
import velovUtils
import pandas as pd
from datetime import timedelta
import datetime
import seaborn as sns
from scipy.stats import exponweib

In [2]:
url = 'https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.json'
conn = psycopg2.connect(database = "velov", 
                        user = "dev", 
                        password = "velov", 
                        host = "localhost" ,
                        port = "5432")

In [3]:
# Create sql statement for retrieve data from database for given station and given hour
station_id = 173
hour = 15
sqlSelect = "select distinct available_bike_stands, available_bikes, last_update_fme from stations_data where station_id =" + str(
        station_id) + " and extract(HOUR FROM last_update_fme) =" + str(hour) + " order by last_update_fme"

In [4]:
# Retrieve all data for station at given hour into a pandas.dataFrame
cur = conn.cursor()
cur.execute(sqlSelect)
data = cur.fetchall()
dataAsDF = pd.DataFrame(data)
labels = ['avail_bike_stands', 'available_bikes', "last_update"]
for columns_label in range(0, len(labels)):
    dataAsDF = dataAsDF.rename(columns={columns_label: labels[columns_label]})

In [ ]:
# Data seems wronf some times. 
# One of the first check to impelments is to ensure that avail_bike_stands + availables_bike = bike_stands

In [5]:
# We must handle all data for identify automatic update all 10 minuts. 
# Algorithm which is applying : If t2 = t1 + 10 minuts +/- e then this update do not correspond to a real velov change but to an automatic update 
# e represent error because the automatic update are not exactly all ten minuts. e = 30 seconds
tmp = list( dataAsDF.last_update[i+1] - dataAsDF.last_update[i] for i in range(0,len(dataAsDF)-1))
TenMinuts = timedelta(minutes = 10)
e = timedelta(seconds = 30)
duration = [timedelta(seconds = 0)]+ tmp
duration = list(duration[i].total_seconds() for i in range(0,len(duration)))
dataAsDF = pd.concat([dataAsDF, pd.DataFrame(duration)], axis=1)

In [6]:
dataAsDF

,avail_bike_stands,available_bikes,last_update,0
0,18,6,2022-05-20 15:02:00,0.0
1,18,6,2022-05-20 15:04:00,120.0
2,18,6,2022-05-20 15:05:02,62.0
3,18,6,2022-05-20 15:08:01,179.0
4,18,6,2022-05-20 15:09:01,60.0
5,18,6,2022-05-20 15:12:12,191.0
6,18,6,2022-05-20 15:14:08,116.0
7,18,6,2022-05-20 15:16:03,115.0
8,19,5,2022-05-20 15:18:08,125.0
9,8,7,2022-05-20 15:20:17,129.0
